In [1]:
# import the proper packages
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Get data
# demand is in a sheet called 'Table1'
demand = pd.read_excel('VertexPetroSystems.xlsx', sheet_name='Table1', index_col=0)
demand

,demand
region,
North_America,12
South_America,8
Europe,14
Asia,16
Africa,7


In [3]:
# convert the demand DataFrame to a dictionary
demand_dict = demand['demand'].to_dict()
demand_dict

{'North_America': 12,
 'South_America': 8,
 'Europe': 14,
 'Asia': 16,
 'Africa': 7}

In [4]:
# Create the plants as a list
plants = list(demand_dict.keys())
plants

['North_America', 'South_America', 'Europe', 'Asia', 'Africa']

In [5]:
# Create the markets as a list
markets = list(demand_dict.keys())
markets

['North_America', 'South_America', 'Europe', 'Asia', 'Africa']

In [6]:
# Get data
# the capacity and fixed costs are in a sheet labeled 'Table3'
capacity_and_fc = pd.read_excel('VertexPetroSystems.xlsx', sheet_name='Table3', index_col=0)
capacity_and_fc

,Low-Capacity,Low-FC,High-Capacity,High-FC
region,,,,
North_America,10,6000,20,9000
South_America,10,4500,20,6750
Europe,10,6500,20,9750
Asia,10,4100,20,6150
Africa,10,4000,20,6000


In [7]:
# Create a dictionary to hold the low capacity for each plant
low_capacity = capacity_and_fc['Low-Capacity'].to_dict()
low_capacity

{'North_America': 10,
 'South_America': 10,
 'Europe': 10,
 'Asia': 10,
 'Africa': 10}

In [8]:
# Create a dictionary to hold the high capacity for each plant
high_capacity = capacity_and_fc['High-Capacity'].to_dict()
high_capacity

{'North_America': 20,
 'South_America': 20,
 'Europe': 20,
 'Asia': 20,
 'Africa': 20}

In [9]:
# Create a dictionary to hold the low fixed cost for each plant
low_fixed_cost = capacity_and_fc['Low-FC'].to_dict()
low_fixed_cost

{'North_America': 6000,
 'South_America': 4500,
 'Europe': 6500,
 'Asia': 4100,
 'Africa': 4000}

In [10]:
# Create a dictionary to hold the high capacity for each plant
high_fixed_cost = capacity_and_fc['High-FC'].to_dict()
high_fixed_cost

{'North_America': 9000,
 'South_America': 6750,
 'Europe': 9750,
 'Asia': 6150,
 'Africa': 6000}

In [11]:
# Get data
# Variable costs are in sheet labeled 'Table2'
var_cost = pd.read_excel('VertexPetroSystems.xlsx', sheet_name='Table2', index_col=0)
var_cost

,North_America,South_America,Europe,Asia,Africa
region,,,,,
North_America,81,92,101,130,115
South_America,117,77,108,98,100
Europe,102,105,95,119,111
Asia,115,125,90,59,74
Africa,142,100,103,105,71


In [12]:
# Convert variable costs to a list of lists
# call it transportation_costs
transportation_costs = var_cost.values.tolist()
transportation_costs

[[81, 92, 101, 130, 115],
 [117, 77, 108, 98, 100],
 [102, 105, 95, 119, 111],
 [115, 125, 90, 59, 74],
 [142, 100, 103, 105, 71]]

In [13]:
# Time to start create the MIP problem
m = gp.Model('VertexPetro')
m.ModelSense = GRB.MINIMIZE

Restricted license - for non-production use only - expires 2026-11-23


In [14]:
# Create the flow variables from plants to markets
flow = m.addVars(plants, markets, obj=transportation_costs, name='flow')

# Update the model
m.update()

In [15]:
# What does the variable flow look like?
flow

{('North_America',
  'North_America'): <gurobi.Var flow[North_America,North_America]>,
 ('North_America',
  'South_America'): <gurobi.Var flow[North_America,South_America]>,
 ('North_America', 'Europe'): <gurobi.Var flow[North_America,Europe]>,
 ('North_America', 'Asia'): <gurobi.Var flow[North_America,Asia]>,
 ('North_America', 'Africa'): <gurobi.Var flow[North_America,Africa]>,
 ('South_America',
  'North_America'): <gurobi.Var flow[South_America,North_America]>,
 ('South_America',
  'South_America'): <gurobi.Var flow[South_America,South_America]>,
 ('South_America', 'Europe'): <gurobi.Var flow[South_America,Europe]>,
 ('South_America', 'Asia'): <gurobi.Var flow[South_America,Asia]>,
 ('South_America', 'Africa'): <gurobi.Var flow[South_America,Africa]>,
 ('Europe', 'North_America'): <gurobi.Var flow[Europe,North_America]>,
 ('Europe', 'South_America'): <gurobi.Var flow[Europe,South_America]>,
 ('Europe', 'Europe'): <gurobi.Var flow[Europe,Europe]>,
 ('Europe', 'Asia'): <gurobi.Var fl

In [16]:
# Display the current state of the model 
m.display()

Minimize
81.0 flow[North_America,North_America] + 92.0 flow[North_America,South_America]
+ 101.0 flow[North_America,Europe] + 130.0 flow[North_America,Asia]
+ 115.0 flow[North_America,Africa] + 117.0 flow[South_America,North_America]
+ 77.0 flow[South_America,South_America] + 108.0 flow[South_America,Europe]
+ 98.0 flow[South_America,Asia] + 100.0 flow[South_America,Africa]
+ 102.0 flow[Europe,North_America] + 105.0 flow[Europe,South_America]
+ 95.0 flow[Europe,Europe] + 119.0 flow[Europe,Asia] + 111.0 flow[Europe,Africa]
+ 115.0 flow[Asia,North_America] + 125.0 flow[Asia,South_America]
+ 90.0 flow[Asia,Europe] + 59.0 flow[Asia,Asia] + 74.0 flow[Asia,Africa]
+ 142.0 flow[Africa,North_America] + 100.0 flow[Africa,South_America]
+ 103.0 flow[Africa,Europe] + 105.0 flow[Africa,Asia] + 71.0 flow[Africa,Africa]
Subject To


/tmp/ipykernel_469/224434616.py:2: DeprecationWarning: Model.display() is deprecated
  m.display()


In [17]:
# Make sure you meet demand for each market
for j in markets:
    m.addConstr(flow.sum('*', j) >= demand_dict[j], name=f'demand_{j}')

m.update()
m.display()

Minimize
81.0 flow[North_America,North_America] + 92.0 flow[North_America,South_America]
+ 101.0 flow[North_America,Europe] + 130.0 flow[North_America,Asia]
+ 115.0 flow[North_America,Africa] + 117.0 flow[South_America,North_America]
+ 77.0 flow[South_America,South_America] + 108.0 flow[South_America,Europe]
+ 98.0 flow[South_America,Asia] + 100.0 flow[South_America,Africa]
+ 102.0 flow[Europe,North_America] + 105.0 flow[Europe,South_America]
+ 95.0 flow[Europe,Europe] + 119.0 flow[Europe,Asia] + 111.0 flow[Europe,Africa]
+ 115.0 flow[Asia,North_America] + 125.0 flow[Asia,South_America]
+ 90.0 flow[Asia,Europe] + 59.0 flow[Asia,Asia] + 74.0 flow[Asia,Africa]
+ 142.0 flow[Africa,North_America] + 100.0 flow[Africa,South_America]
+ 103.0 flow[Africa,Europe] + 105.0 flow[Africa,Asia] + 71.0 flow[Africa,Africa]
Subject To
demand_North_America: flow[North_America,North_America] +
flow[South_America,North_America] + flow[Europe,North_America] +
 flow[Asia,North_America] + flow[Africa,North_Am

/tmp/ipykernel_469/907243498.py:6: DeprecationWarning: Model.display() is deprecated
  m.display()


In [18]:
# Create the binary variables to open low or high capacity plants
for i in plants:
    m.addVar(vtype=GRB.BINARY, obj=low_fixed_cost[i], name=f'y_low_{i}')
    m.addVar(vtype=GRB.BINARY, obj=high_fixed_cost[i], name=f'y_high_{i}')

m.update()
m.display()

Minimize
81.0 flow[North_America,North_America] + 92.0 flow[North_America,South_America]
+ 101.0 flow[North_America,Europe] + 130.0 flow[North_America,Asia]
+ 115.0 flow[North_America,Africa] + 117.0 flow[South_America,North_America]
+ 77.0 flow[South_America,South_America] + 108.0 flow[South_America,Europe]
+ 98.0 flow[South_America,Asia] + 100.0 flow[South_America,Africa]
+ 102.0 flow[Europe,North_America] + 105.0 flow[Europe,South_America]
+ 95.0 flow[Europe,Europe] + 119.0 flow[Europe,Asia] + 111.0 flow[Europe,Africa]
+ 115.0 flow[Asia,North_America] + 125.0 flow[Asia,South_America]
+ 90.0 flow[Asia,Europe] + 59.0 flow[Asia,Asia] + 74.0 flow[Asia,Africa]
+ 142.0 flow[Africa,North_America] + 100.0 flow[Africa,South_America]
+ 103.0 flow[Africa,Europe] + 105.0 flow[Africa,Asia] + 71.0 flow[Africa,Africa]
+ 6000.0 y_low_North_America + 9000.0 y_high_North_America + 4500.0 y_low_South_America
+ 6750.0 y_high_South_America + 6500.0 y_low_Europe + 9750.0 y_high_Europe
+ 4100.0 y_low_Asia

/tmp/ipykernel_469/1745217750.py:7: DeprecationWarning: Model.display() is deprecated
  m.display()


In [19]:
# Create capacity constraints being sure you stay under capacity
for i in plants:
    m.addConstr(flow.sum(i, '*') <= low_capacity[i]*m.getVarByName(f'y_low_{i}') +
                high_capacity[i]*m.getVarByName(f'y_high_{i}'),
               name=f'capacity_{i}')

m.update()
m.display()

Minimize
81.0 flow[North_America,North_America] + 92.0 flow[North_America,South_America]
+ 101.0 flow[North_America,Europe] + 130.0 flow[North_America,Asia]
+ 115.0 flow[North_America,Africa] + 117.0 flow[South_America,North_America]
+ 77.0 flow[South_America,South_America] + 108.0 flow[South_America,Europe]
+ 98.0 flow[South_America,Asia] + 100.0 flow[South_America,Africa]
+ 102.0 flow[Europe,North_America] + 105.0 flow[Europe,South_America]
+ 95.0 flow[Europe,Europe] + 119.0 flow[Europe,Asia] + 111.0 flow[Europe,Africa]
+ 115.0 flow[Asia,North_America] + 125.0 flow[Asia,South_America]
+ 90.0 flow[Asia,Europe] + 59.0 flow[Asia,Asia] + 74.0 flow[Asia,Africa]
+ 142.0 flow[Africa,North_America] + 100.0 flow[Africa,South_America]
+ 103.0 flow[Africa,Europe] + 105.0 flow[Africa,Asia] + 71.0 flow[Africa,Africa]


/tmp/ipykernel_469/669566540.py:8: DeprecationWarning: Model.display() is deprecated
  m.display()


+ 6000.0 y_low_North_America + 9000.0 y_high_North_America + 4500.0 y_low_South_America
+ 6750.0 y_high_South_America + 6500.0 y_low_Europe + 9750.0 y_high_Europe
+ 4100.0 y_low_Asia + 6150.0 y_high_Asia + 4000.0 y_low_Africa + 6000.0 y_high_Africa
Subject To
demand_North_America: flow[North_America,North_America] +
flow[South_America,North_America] + flow[Europe,North_America] +
 flow[Asia,North_America] + flow[Africa,North_America] >= 12
demand_South_America: flow[North_America,South_America] +
flow[South_America,South_America] + flow[Europe,South_America] +
 flow[Asia,South_America] + flow[Africa,South_America] >= 8
demand_Europe: flow[North_America,Europe] + flow[South_America,Europe] +
 flow[Europe,Europe] + flow[Asia,Europe] + flow[Africa,Europe] >= 14
demand_Asia: flow[North_America,Asia] + flow[South_America,Asia] + flow[Europe,Asia] +
 flow[Asia,Asia] + flow[Africa,Asia] >= 16
demand_Africa: flow[North_America,Africa] + flow[South_America,Africa] +
 flow[Europe,Africa] + flow[

In [20]:
# Solve the model
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: AMD EPYC 7571, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 10 rows, 35 columns and 60 nonzeros
Model fingerprint: 0xf5c42200
Variable types: 25 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 2e+01]
Presolve time: 0.00s
Presolved: 10 rows, 35 columns, 60 nonzeros
Variable types: 25 continuous, 10 integer (10 binary)
Found heuristic solution: objective 67039.000000

Root relaxation: objective 2.277150e+04, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 22771.5000    0    1 67039.0000 22771.5000  66.0%     -    

In [21]:
# Print out the results
print(f'Total Cost: ${m.ObjVal:,.2f}')
for v in m.getVars():
    if v.X > 0:
        print(f'{v.VarName} = {v.X}')

Total Cost: $23,751.00
flow[South_America,North_America] = 12.0
flow[South_America,South_America] = 8.0
flow[Asia,Europe] = 4.0
flow[Asia,Asia] = 16.0
flow[Africa,Europe] = 10.0
flow[Africa,Africa] = 7.0
y_high_South_America = 1.0
y_high_Asia = 1.0
y_high_Africa = 1.0
